<a href="https://colab.research.google.com/github/Ymin-2/ESAA/blob/main/ESAA_YB_WEEK07_2_review1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**주제**
인체 내 약물 대사에 관여하는 CYP3A4 효소 저해 예측모델 개발

##**데이터**

* train.csv
  - CYP3A4 효소 저해 관련 데이터 1,681종
  - ID : 고유 ID
  - Canonical_Smies : 분자구조 데이터
  - Inhibition : 저해율(단위 : %)

* test.csv
  - ID : 고유 ID
  - Canonical_Smies : 분자구조 데이터

* sample_submission.csv
  - ID : 고유 ID
  - Inhibition : 저해율(단위 : %)

##**코드 리뷰**

1. 데이터 준비
- SMILES(분자 구조 표현)만 남겨 train_cp, test_cp로 저장
  - chemprop 명령줄 툴이 이 CSV를 입력으로 사용
2. 데이터 증강 함수(SMILES 랜덤화)
- def get_augmented_smiles
3. MPNN 모델 3종 학습
- 3가지 구성의 MPNN(Message Passing Neural Network) 모델 학습
- 서로 다른 hidden_size와 depth로 다양성 확보로 앙상블 효과 기대
4. TTA(Test Time Augmentation)
- 테스트 시에도 증강된 SMILES 여러 개를 만들어 예측 후 평균냄
- TTA를 통해 예측 안정화(분자 표현 불확실성 완화)
5. MPNN 예측 결과를 LightGBM 입력으로 사용
- 각 모델의 예측값을 모아 테이블 구성
  - smiles, pred_meta_st_0, pred_meta_st_1, pred_meta_st_2
- 이게 LightGBM의 feature로 들어감
6. LightGBM 메타 모델 학습
- X_meta: 3개 MPNN 예측값들
- y_meta: 실제 저해율(Inhibition)
- 3개의 MPNN 결과를 통합해 최적 가중 조합 학습
7. 상·하위 25% 예측값 보정
- 예측값이 상위 25% → 5% 더 높게 보정
- 예측값이 하위 25% → 5% 더 낮게 보정
- 극단값 강조 보정으로 모델 예측의 스케일 보완
8. 최종 제출
- LightGBM 결과를 보정 후 제출 형식에 맞게 저장

##**차별 및 배울점**

- 단일 모델이 아닌 딥러닝(MPNN)과 LightGBM 결합 구조 (앙상블)
  - LightGBM은 비선형 관계 학습과 과적합 제어가 쉬움
  - 여러 MPNN의 결과를 조합할 때 유리
- 텍스트 데이터를 랜덤하게 표현 바꾸기로 확장함(get_augmented_smiles)
- TTA의 개념을 새롭게 배움
  - 테스트 시에도 여러 버전으로 예측 후 평균내는 전략